## 07_forced_aligner

## What is a forced aligner?
"Forced aligner" is the name for software that automatically aligns audio with a text file, usually through phonetic transcription. Forced aligners accomplish this through some sort of auditory model: they know what certain phonemes are expected to sound like, then make predictions on the locations of those phonemes using a user-provided audio file and accompanying transcript.

Some examples:

* [Penn Phonetics Forced Aligner (P2FA)](https://babel.ling.upenn.edu/phonetics/old_website_2015/p2fa/index.html)
* [FAVE-align (technically based on P2FA but whatever)](https://github.com/JoFrhwld/FAVE/wiki/FAVE-align)'
* [Gentle](http://lowerquality.com/gentle/)
* [Montreal Forced Aligner](https://montreal-forced-aligner.readthedocs.io/en/latest/)

In this notebook we will be using P2FA, because in the Hamilton Lab we either use FAVE or P2FA and I'm more familiar with P2FA (I also think it works better, but that's just my opinion...)

Both are Python-importable, so let's do that.

In [1]:
from supplemental_files.PennPhoneticsForcedAligner.align import main as p2fa

This function, `main()` takes four arguments:

* `wavfile` : the path to your .wav file
* `trsfile` : the path to your transcript (.txt) file
* `outfile` : the output location where you want your .textgrid file
* `mypath` : this one is kinda weird, but this is the path to the "model" folder within the PennPhoneticsForcedAligner file. If you are running the forced aligner directly from this folder (PennPhoneticsForcedAligner), then you don't need to put this as an argument (the default value assumes youre in the folder), but otherwise I recommend using an absolute path to this folder. So for this notebook it would be: `./supplemental_files/PennPhoneticsForcedAligner/model`

## Getting your .wav file together
Seems easy enough right? Give it audio, give it a text file, boom you're done? Well, not exactly. The audio file needs to be in one of three bitrates, which are dictated by the available underlying HTK models that P2FA uses: 8000Hz, 11025Hz, and 16000Hz.

I personally have had the most success with 11025 Hz, but if one isn't working well for a particular wavfile, try resampling it and try again! Note that there is a processing speed tradeoff: a higher sampling rate will need more time to align. So if you are on a crappy computer, maybe start with 8000Hz and work your way up from there.

Need help resampling audio? Check out [06_preprocessing_audio](https://github.com/HamiltonLabUT/lab_intro_notebooks/blob/main/06_preprocessing_audio.ipynb).

## Getting your transcript file together

**Your transcript file needs to perfectly match your wav file. If it doesn't, you will have problems.** This is not an issue for fixed-stimulus speech perception experiments (for example, Maansi's TIMIT task). For speech production tasks however, you will need to make sure that the transcript matches exactly what the subject said, errors and all. There are some useful shorthands used by P2FA that you can include in your transcription file as well:

* `{NS}` - background noise
* `{CG}` - cough
* `{LG}` - laughter
* `{BR}` - breath
* `{SL}` - silence
* `{LS}` - lipsmack
* `sp` - small pause

If the forced aligner is having difficulty with certain parts of your audio, try playing around with these labels. It may help performance.

## Let's run the forced aligner on an example transcript
For this step, I want you to open up Praat and record yourself (New -> Record Mono Sound) saying the following:

```
According to all known laws of aviation, there is no way a bee should be able to fly.
Its wings are too small to get its fat little body off the ground.
The bee, of course, flies anyway because bees don't care what humans think is impossible.
```

**Make sure you set your sampling frequency to 11025 Hz in the Praat recording interface!**

I have provided a transcript file of this in `./supplemental_files/example_transcript.txt`.

## oh also we need to install HTK
HTK is the speech recognition toolkit that P2FA runs on. You can download it from [their website](https://htk.eng.cam.ac.uk/) (you need to register but it's free and they literally have never sent me an email).

Installation guide for OSX: https://wstyler.ucsd.edu/posts/p2fa_mac.html

Installation guide for Windows: https://htk.eng.cam.ac.uk/docs/inst-win.shtml

In [2]:
# Set up arguments for forced aligner
trsfile = "./supplemental_files/example_transcript.txt"
# update this path with your .wav you recorded in Praat:
wavfile = "/Users/kfsh/Desktop/example_audio.wav"
outfile = "./example_textgrid.textgrid" # save to ..git/lab_intro_notebooks/
mypath = "./supplemental_files/PennPhoneticsForcedAligner/model"

In [3]:
# Run main() (i set the alias as p2fa() on import)
p2fa(wavfile, trsfile, outfile, mypath)

Using wav file, already at sampling rate 11025.
11025
Running HVite...
./example_textgrid.textgrid


## Troubleshooting, tips, and tricks
These are adapted from a notebook I wrote for my [Masters thesis](https://github.com/HamiltonLabUT/onsetProd/blob/master/transcription/transcriptionPipeline.ipynb).

#### Long audio files and processing speed of P2FA
The length of the wavfile has a non-linear relationship with how long the forced aligner takes to run. That means processing speed _exponentially_ decreases with wavfile length. A quick fix to this is to split your wavfile/transcripts up into manageable chunks and run those individually, then splice the textgrid back together at the end (you can do this in the Praat GUI by selecting multiple TextGrids and clicking "Concatenate"). 10 minute chunks (600s) work well for me. If you do this, beware that sometimes P2FA will put the start in the textgrid at 0.0125 instead of 0, which will cause offset if you are chaining textgrids back together. Talk to Garret if you want help doing this - an undergrad alum of the lab wrote some code for handling this bug.

#### Issues with OSX installation
If you are on OS X, I suggest following this guide: http://wstyler.ucsd.edu/posts/p2fa_mac.html

* During one installation, for some godforsaken reason, downloading the command line tools did not install X11. If your Terminal during `make all` is whining about X11, download and install it from this link: https://www.xquartz.org/
* Here is another okay guide for installation on OS X in case something from my suggested guide is not working: https://github.com/JoFrhwld/FAVE/wiki/HTK-on-OS-X

#### Common errors while running `main()` from `align.py`
```
~/Desktop/git/onsetProd/transcription/PennPhoneticsForcedAligner/align.py in readAlignedMLF(mlffile, SR, wave_start)
    133 
    134     if len(lines) < 3 :
--> 135         raise ValueError("Alignment did not complete succesfully.")
    136 
    137     j = 2

ValueError: Alignment did not complete succesfully.
```
**How to fix it**: I fixed this by changing the sampling rate on my .wav file. I suspect dictionary issues may also cause this error though (ie., too many words are skipped).

<br/>

```
FileNotFoundError: [Errno 2] No such file or directory: './tmp/aligned.mlf'
```
**How to fix it**: There's two reasons this error code happens that I've found:
1. The forced aligner can't find HVite. To fix this, you have to edit the path to HVite in line 229 of `align.py`. If you want to know what path to use, you can run `which HVite` in your Terminal.
2. You are using the wrong version of HTK. Using the most recent build (3.4.1) breaks the forced aligner. You need to use 3.4. To check what version of HTK you are using, go to your Terminal and type `HVite -V`.

<br/>

Lastly, if your textgrid is created but the alignment is super wack, check your transcript! It's probably off.

<br/>

There are a few other error codes that are not accessible from this notebook -- you have to run `align.py` in Terminal to see the full output:

* `sh: /HVite: No such file or directory` -- I was able to fix this by pointing `align.py` to the correct path for HVite (see above).

* `ERROR [+8522]  LatFromPaths: Align have dur<=0 ` -- I was able to fix this by changing my HTK from 3.4.1 to 3.4 (see above).

#### If nothing you try is working, try resampling.
* You can change the sampling rate of your .wav file. The forced aligner likes 11025Hz in particular. You can do this by running the following line in your Terminal: `sox input_file.wav -r 11025 output_file.wav`
* Make sure you have sox and Hvite installed by typing these in your Terminal: `which sox` & `which HVite`


#### If your TextGrids are being created successfully, but look like trash
The ForcedAligner did a horrible job and needs to be tweaked.

Here are some things you can try before re-running `main()` (roughly in order):
* Re-preproces your audio
* High-pass filter your audio at 10Hz -- sometimes there's a weird DC shift that P2FA doesn't like. This comes from recording our audio on an EEG amplifier.
* Try sampling at 16kHz (c.f. 11025 Hz). Or try 8000 Hz.
* Do noise reduction in Audacity, or other free audio editing software. Just take particiularly noisy parts and silence them. Quick Google tutorial: https://kb.plu.edu/page.php?id=86087

Things I've never tried before but like the idea of trying if nothing else helps:
* Normalize the waveform so that the max amplitude is 1: `audio = audio.max()`.
* insert `sp` or `{NS}` after each sentence in the transcript files (This might be good to do anyways).
* look at Section E ("Doing Forced Alignment The Hard Way") of the P2FA documentation -- it may be possible to re-train their HMM on already-aligned textgrids from your task. (This sounds really hard and I personally would save this for a last resort)

IF ALL ELSE FAILS, you will have to change how transcription is done. This is really tedious and I wouldn't suggest unless you have exhausted all other options (including trying alternative forced aligners), but you could:
* Split the .wav file and transcript files sentence-by-sentence, so that each textgrid is only one sentence long
* Explicitly pass onset/offset times as `-s start_time` and `-e end_time` into the forced aligner. This means you will have to run the FA for each sentence in your block (aka a lot of times), also getting the onset/offset times will be a lot of manual labor.